Thomas Dougherty

Probability and Statistics for Computer Science


Introduction

    To Do:

    Figure out script to iterate through and cleanup all CSVs

    Store data from all CSVs

    Aggregate data from *all* CSVs into one dataframe
    

### March 2018

In [7]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper_functions import *

#read CSV for March 2018
services = pd.read_csv('data\\2018_03.csv')
#status column is not needed so that will be dropped
services.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services.drop(services[services['type'] == 'Amtrak'].index, inplace=True)
services.astype({'date' : 'datetime64[ns]',
                'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float16',
                'line' : 'category',
                'type' : 'category'})
services.dtypes
services.head(10)

,date,train_id,stop_sequence,from,from_id,to,to_id,scheduled_time,actual_time,delay_minutes,line,type
0,2018-03-01,3805,1.0,New York Penn Station,105,New York Penn Station,105,2018-03-02 01:22:00,2018-03-02 01:21:05,0.000000,Northeast Corrdr,NJ Transit
1,2018-03-01,3805,2.0,New York Penn Station,105,Secaucus Upper Lvl,38187,2018-03-02 01:31:00,2018-03-02 01:31:08,0.133333,Northeast Corrdr,NJ Transit
2,2018-03-01,3805,3.0,Secaucus Upper Lvl,38187,Newark Penn Station,107,2018-03-02 01:40:00,2018-03-02 01:40:07,0.116667,Northeast Corrdr,NJ Transit
3,2018-03-01,3805,4.0,Newark Penn Station,107,Newark Airport,37953,2018-03-02 01:45:00,2018-03-02 01:45:10,0.166667,Northeast Corrdr,NJ Transit
4,2018-03-01,3805,5.0,Newark Airport,37953,North Elizabeth,109,2018-03-02 01:49:00,2018-03-02 01:49:10,0.166667,Northeast Corrdr,NJ Transit
5,2018-03-01,3805,6.0,North Elizabeth,109,Elizabeth,41,2018-03-02 01:52:00,2018-03-02 01:52:01,0.016667,Northeast Corrdr,NJ Transit
6,2018-03-01,3805,7.0,Elizabeth,41,Linden,70,2018-03-02 01:58:00,2018-03-02 01:58:05,0.083333,Northeast Corrdr,NJ Transit
7,2018-03-01,3805,8.0,Linden,70,Rahway,127,2018-03-02 02:02:00,2018-03-02 02:01:03,0.000000,Northeast Corrdr,NJ Transit
8,2018-03-01,3805,9.0,Rahway,127,Metropark,83,2018-03-02 02:08:00,2018-03-02 02:08:00,0.000000,Northeast Corrdr,NJ Transit
9,2018-03-01,3805,10.0,Metropark,83,Metuchen,84,2018-03-02 02:13:00,2018-03-02 02:13:10,0.166667,Northeast Corrdr,NJ Transit


In [8]:
 #create new dataframe with data broken down by line
# function to create df. figure out how to import pandas into helper file???
def create_new_dataframe(list_njt_lines, list_max_delays, list_avg_delays, list_dates):
    df = pd.DataFrame({'Longest Delay (minutes)': list_max_delays.values,
                                   'Average Delay (minutes)': list_avg_delays.values,
                                   'Date of Longest Delay' : list_dates.values},
                                    index=list_njt_lines)
    return df

njt_lines=services['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services,njt_lines))
train_ids = pd.Series(helper.get_train_ids(services,njt_lines))

performance_by_line_march = create_new_dataframe(njt_lines, max_delays, avg_delays, dates)

performance_by_line_march.astype({'Longest Delay (minutes)' :'float16',
                            'Average Delay (minutes)' :'float16',
                            'Date of Longest Delay' : 'datetime64[ns]'
                            })

performance_by_line_march

,Longest Delay (minutes),Average Delay (minutes),Date of Longest Delay
Northeast Corrdr,91.083,4.253,2018-03-12
No Jersey Coast,100.267,3.932,2018-03-01
Main Line,86.000,3.542,2018-03-07
Morristown Line,100.283,3.780,2018-03-02
Gladstone Branch,99.083,3.654,2018-03-02
Raritan Valley,94.000,2.662,2018-03-07
Bergen Co. Line,79.000,2.964,2018-03-07
Atl. City Line,100.217,4.482,2018-03-07
Montclair-Boonton,111.183,3.382,2018-03-07
Princeton Shuttle,4.467,0.096,2018-03-04


#### Analysis:
##### Most of the delays occur between March 1 and March 7. In 2018, two severe winter storms travelled across the eastern United States. One on March 1-2 and March 6-8 just a few days later. The storms brought high wind gusts, precipation, and coastal flooding leading to service delays and cancellations.

###### https://www.njtransit.com/press-releases/nj-transit-implements-severe-weather-rail-schedule-winter-storm

###### https://www.climate.gov/news-features/event-tracker/nor%E2%80%99easters-pummel-us-northeast-late-winter-2018

### November 2018:

In [9]:
#read CSV for March 2018
services = pd.read_csv('data\\2018_11.csv')
#status column is not needed so that will be dropped
services.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services.drop(services[services['type'] == 'Amtrak'].index, inplace=True)
services.astype({'date' : 'datetime64[ns]',
                'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float16',
                'line' : 'category',
                'type' : 'category'})
services.dtypes
services.head(10)

# create new df
njt_lines=services['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services,njt_lines))
train_ids = pd.Series(helper.get_train_ids(services,njt_lines))

performance_by_line_nov = create_new_dataframe(njt_lines, max_delays, avg_delays, dates)

print(performance_by_line_nov)


IndexError: index 0 is out of bounds for axis 0 with size 0